In [ ]:
#task one 
import numpy as np
import pandas as pd
import cv2

def get_color_value(hue_value):
    color = "Undefined"
    if hue_value < 5:
        color = "RED"
    elif hue_value < 22:
        color = "ORANGE"
    elif hue_value < 33:
        color = "YELLOW"
    elif hue_value < 78:
        color = "GREEN"
    elif hue_value < 131:
        color = "BLUE"
    elif hue_value < 170:
        color = "VIOLET"
    else:
        color = "RED"
    return color    
def get_frame_centers(frame):
    cx = int(width/2)
    cy = int (height/2)
    return cx,cy       
cap = cv2.VideoCapture(0)
while True : 
    _, frame= cap.read()
    height, width, _ = frame.shape
    image = frame.copy()
    cx,cy = get_frame_centers(frame)
    #get color image
    hsv_image    = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    center_of_one_pixel = hsv_image[cy,cx]
    hue_value    = center_of_one_pixel[0]  
    color = get_color_value(hue_value)     
   #get same data in main frame 
    main_center_pixel = image[cy,cx]
    b,g,r = int(main_center_pixel[0]),int(main_center_pixel[1]),int(main_center_pixel[2])
    cv2.putText(image, color, (cx - 300, 20), 0, .5, (b, g, r), 2)
    cv2.circle(image,(cx,cy),3,(0, 255, 255),3)  
    cv2.imshow('image',image)  
    key = cv2.waitKey(1)  & 0xFF  
    if key == ord('q'):
        cap.release()
        cv2.destroyAllWindows()      

In [29]:
import numpy as np
import pandas as pd
import cv2
import os

class Reinhard_Color_Transfer:
    def __init__(self, img_path,template_path):
            self.img_path = img_path 
            self.template_path=template_path
            
     #define image to get mean of imagesand stander deviation 
    def get_mean_and_std(self,image):
            x_mean, x_std = cv2.meanStdDev(image)
            x_mean = np.hstack(np.around(x_mean,2))
            x_std  = np.hstack(np.around(x_std,2))           
            return x_mean,x_std
        #load images for  inputs images
    def load_images(self) :
        all_images=[]
        for image in os.listdir(self.img_path):
            path = os.path.join(self.img_path, image)
            image = cv2.imread(path)
            image = cv2.cvtColor(image,cv2.COLOR_BGR2LAB)
            all_images.append(image)
            return all_images
        
    def load_template(self)  :
        alltemplates=[]
        for template in os.listdir(self.template_path):
            path = os.path.join(self.template_path, template)
            template = cv2.imread(path)
            template = cv2.cvtColor(template,cv2.COLOR_BGR2LAB)
            alltemplates.append(template)
            return alltemplates
        
        
    def start_transfer (self):
        #start load inputs and template images
        input_images    = self.load_images()
        template_images = self.load_template()
        for i ,template in enumerate(template_images):
            #calculate std,mean
            template_mean,template_std = self.get_mean_and_std(template)
            #load inpu images
            for j , input_image in  enumerate(input_images):
                    #calculate std and template 
                    image_mean,image_std = self.get_mean_and_std(input_image)
                    height,width,channel = input_image.shape   
                    for x in range(0,height):
                        for y in range(0,width):
                            for z in range(0,channel):        
                            #get pixel by pixel 
                                img=input_image[x,y,z]
                                img = ((img - image_mean[z]) * (template_std[z] / image_std[z])) + template_mean[z] 
                                img = round(img) 
                                img = 0 if img<0 else img
                                img = 255 if img > 255 else img
                                input_image [x,y,z]=img     
                    input_image = cv2.cvtColor(input_image, cv2.COLOR_LAB2BGR)
                    cv2.imwrite(f"c:/Users/hhhh/Desktop/oput/{i+1}_input{j+1}.jpg", input_image)

image_path = 'c:/Users/hhhh/Desktop/in/'
template_path = 'c:/Users/hhhh/Desktop/temp/'
transfer_object =Reinhard_Color_Transfer(image_path, template_path)
# Transfer the images
transfer_object.start_transfer()                    

In [92]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os

class Thresholding:
        def __init__(self,path):
            self.path = path
            
        def load_images(self) :
            all_images=[]
            for image in os.listdir(self.path):
                image = cv2.imread(os.path.join(self.path,image))
                all_images.append(image)
            return all_images
        def imageGray(self,img):
            return cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
                
        def apply_Gussian_Buller(self,img):
            return cv2.GaussianBlur(img,(3,3),0)
        
        def apply_thresh_binary(self,img):
            _,th1 = cv2.threshold(img,120,255,cv2.THRESH_BINARY)
            return th1
        def apply_thresh_binary_inverse(self,img):
            _,th1 = cv2.threshold(img,120,255,cv2.THRESH_BINARY_INV)
            return th1
        
        def apply_adaptive_mean(self,img):
            th1 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C ,cv2.THRESH_BINARY,11,2)
            return th1
      
        def apply_adaptive_gaussian(self,img):
            th1 =cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
            return th1 
        
        def apply_otsu(self,img):
            _,otsu = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
            return otsu
        
        def start_thresholding (self):
            images = self.load_images()
            for i ,image in enumerate(images):
                gray_image=self.imageGray(image)
                image_buler =self.apply_Gussian_Buller(gray_image) 
                thresholdsh_binary=self.apply_thresh_binary(image_buler)
                thresh_binary_inverse=self.apply_thresh_binary_inverse(image_buler)
                adaptive_gaussian=self.apply_adaptive_gaussian(image_buler)
                adaptive_mean=self.apply_adaptive_mean(image_buler)
                otsu=self.apply_otsu(image_buler)
                cv2.imwrite(f'c:/Users/hhhh/Desktop/out_thresh/thresholdsh_binary_{i}.jpg',thresholdsh_binary)
                cv2.imwrite(f'c:/Users/hhhh/Desktop/out_thresh/thresh_binary_inverse_{i}.jpg',thresh_binary_inverse)
                cv2.imwrite(f'c:/Users/hhhh/Desktop/out_thresh/thresholdsh_binary_{i}.jpg',thresholdsh_binary)
                cv2.imwrite(f'c:/Users/hhhh/Desktop/out_thresh/adaptive_gaussian_{i}.jpg',adaptive_gaussian)
                cv2.imwrite(f'c:/Users/hhhh/Desktop/out_thresh/adaptive_mean_{i}.jpg',adaptive_mean)
                cv2.imwrite(f'c:/Users/hhhh/Desktop/out_thresh/otsu_{i}.jpg',otsu)
                
image_path = 'C:/Users/hhhh/Desktop/in_thresh/'
thres = Thresholding(image_path)
thres.start_thresholding()
print('images Created Successfuly')
            

images Created Successfuly


In [2]:
import numpy as np
import pandas as pd
import cv2


In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os


class Morphological :
    def __init__(self,path):
        self.path = path
        
    def load_images(self) :
        all_images=[]
        for image in os.listdir(self.path):
            image = cv2.imread(os.path.join(self.path,image))
            all_images.append(image)
        return all_images

    def image_preprocessing (self,image):
        gray_image   = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        buller_image = cv2.GaussianBlur(gray_image,(3,3),0)
        _,thres_image = cv2.threshold(buller_image,50,255,cv2.THRESH_BINARY)
        return thres_image
    
    def apply_dilation(self,image):
        kernel = np.ones((5,5),np.uint8)
        return cv2.dilate(image,kernel,iterations= 2)
    
    def apply_errotion(self,image):
        kernel = np.ones((5,5),np.uint8)
        return cv2.erode(image,kernel,iterations= 2)
    def apply_oppening(self,image):
        kernel = np.ones((5,5),np.uint8)
        return cv2.morphologyEx(image,cv2.MORPH_OPEN,kernel,iterations=2)
    def apply_closing(self,image):
        kernel = np.ones((5,5),np.uint8)
        return cv2.morphologyEx(image,cv2.MORPH_CLOSE,kernel,iterations=2)
        
    def start_morph(self):
        images = self.load_images()
        for i ,image in enumerate(images):
            image_thresh = self.image_preprocessing(image)
            dilated      = self.apply_dilation(image_thresh)
            erroded      = self.apply_errotion(image_thresh)
            openning      = self.apply_oppening(image_thresh)
            clothing      = self.apply_closing(image_thresh)
            cv2.imwrite(f'c:/Users/hhhh/Desktop/out_morph/dilated_{i}.jpg',dilated)
            cv2.imwrite(f'c:/Users/hhhh/Desktop/out_morph/erroded_{i}.jpg',erroded)
            cv2.imwrite(f'c:/Users/hhhh/Desktop/out_morph/openning_{i}.jpg',openning)
            cv2.imwrite(f'c:/Users/hhhh/Desktop/out_morph/clothing{i}.jpg',clothing)

image_path = 'C:/Users/hhhh/Desktop/in_morph/'
morph = Morphological(image_path)
morph.start_morph()
print('images Created Successfuly')
        

images Created Successfuly


In [ ]:
start